In [29]:
#Package imported

from pyspark.sql.functions import expr
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,FloatType
from pyspark.sql.functions import from_json,col

In [ ]:
#Initialize the spark session
spark = ( 
    SparkSession
    .builder
    .master("local[*]")
    .appName("Spark streaming from kafka")
    .config("spark.streaming.stopGracefulluOnShutdown", True)
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0')
    .config("spark.sql.shufflle.partitions",1)
    .getOrCreate()
)
    

In [30]:
#Set the kafka readStream
kafka_df =(
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers","ed-kafka:29092")
    .option("subscribe", "streaming")
    .option("startingoffsets", "earliest")
    .load()
)

In [31]:
#Parse the value of the stream data from binary to String

kafka_json_df= kafka_df.withColumn("value",expr("cast(value as string)"))

In [32]:
#Set the json_schema

json_schema=(
    StructType(
        [StructField('Price',FloatType(),True),
         StructField('Volume',FloatType(), True),
         StructField('TrailingPE',FloatType(), True)]))
    
        
        

In [33]:
#Apply the json_schema to the value_json of the data

streaming_df=kafka_json_df.withColumn("value_json", from_json(col("value"),json_schema)).selectExpr("value_json.*")

In [40]:
#Function for writing the stream data to streaming.parquet

def write_par(df,batch_id):
    df.write.format("parquet").mode("append").save("data/streaming.parquet/")

In [ ]:
#Use writeStream to write the streaming data to streaming.parquet

streaming_df.writeStream.foreachBatch(write_par).trigger(processingTime="10 seconds").option("checkpointLocation","check_point_dir_kafka").start().awaitTermination()

In [ ]:
#Use writeStream to write the streaming data to console

streaming_df.writeStream.format("console").outputMode("append").option("checkpointLocation","check_point_dir_kafka").start().awaitTermination()